In [60]:
# from google.colab import drive
# drive.mount('/content/drive')

In [34]:
!pip install ultralytics
!pip install roboflow
!pip install supervision


In [35]:
# from ultralytics import YOLO

# model = YOLO("yolov8x")

In [36]:
# results=model.predict('/content/08fd33_4.mp4',save =True) # saving the result

In [37]:
# print(results[0])

In [38]:
# for box in results[0].boxes:
#   print(box)

In [39]:
from roboflow import Roboflow
rf = Roboflow(api_key="vsIbG6bZsuJgyA2iHsDA") #api key by roboflow
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [40]:
# import shutil #moving to avoid crash
# shutil.move('/content/football-players-detection-1/valid','/content/football-players-detection-1/football-players-detection-1/valid')
# shutil.move('/content/football-players-detection-1/test','/content/football-players-detection-1/football-players-detection-1/test')
# shutil.move('/content/football-players-detection-1/train','/content/football-players-detection-1/football-players-detection-1/train')


In [41]:
# traning
# !yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

In [42]:
# from ultralytics import YOLO

# model = YOLO("/content/drive/MyDrive/football analysis folder/runs/detect/train3/weights/best.pt")  # using trained model

In [43]:
# results=model.predict('/content/08fd33_4.mp4',save =True)

In [44]:
# print(results[0])

In [45]:
# for box in results[0].boxes:
  # print(box)

In [46]:
from ultralytics import YOLO
import supervision as sv
import pickle
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.cluster import KMeans

In [47]:

video_path = "/content/input video/input.mp4"
model_path = "/content/model/best.pt"
stub_path = "/content/stub/track-stubs.pkl"

In [48]:
import hashlib

def get_video_hash(video_path):
    hasher = hashlib.md5()

    with open(video_path, "rb") as f:
        while chunk := f.read(8192):
            hasher.update(chunk)

    return hasher.hexdigest()


In [49]:
import matplotlib.pyplot as plt

def draw_top_down_field(tracks, frame_num):
    """
    Draw a top-down tactical view of the football field for a given frame.
    Uses real-world transformed positions.
    """

    # Standard football pitch dimensions (meters)
    PITCH_LENGTH = 105
    PITCH_WIDTH = 68

    # Safety check
    if frame_num >= len(tracks["players"]):
        raise ValueError("Frame number out of range")

    fig, ax = plt.subplots(figsize=(12, 8))

    #Draw pitch rectangle
    ax.add_patch(
        plt.Rectangle(
            (0, 0),
            PITCH_LENGTH,
            PITCH_WIDTH,
            fill=False,
            edgecolor="black",
            linewidth=2
        )
    )

    #Draw center line
    ax.plot([PITCH_LENGTH / 2, PITCH_LENGTH / 2], [0, PITCH_WIDTH], color="black")

    #Draw center circle
    center_circle = plt.Circle(
        (PITCH_LENGTH / 2, PITCH_WIDTH / 2),
        9.15,
        color="black",
        fill=False
    )
    ax.add_patch(center_circle)


    #Players (colored by team)
    for track_id, player in tracks["players"][frame_num].items():
        pos = player.get("position_transformed")
        team = player.get("team")

        if pos is None or team is None:
            continue

        if team == 1:
            color = "red"
        elif team == 2:
            color = "blue"
        else:
            color = "gray"

        ax.scatter(pos[0], pos[1], c=color, s=60)
        ax.text(pos[0] + 0.5, pos[1] + 0.5, str(track_id), fontsize=8)

    #Referees
    for _, referee in tracks["referees"][frame_num].items():
        pos = referee.get("position_transformed")
        if pos is None:
            continue
        ax.scatter(pos[0], pos[1], c="yellow", s=60)

    # Ball
    ball = tracks["ball"][frame_num].get(1)
    if ball:
        pos = ball.get("position_transformed")
        if pos is not None:
            ax.scatter(pos[0], pos[1], c="white", edgecolors="black", s=120)

    #Formatting
    ax.set_xlim(0, PITCH_LENGTH)
    ax.set_ylim(0, PITCH_WIDTH)
    ax.set_aspect("equal")
    ax.set_title(f"Tactical Top-Down View – Frame {frame_num}")
    ax.axis("off")

    plt.show()


In [50]:
class CameraMovementEstimator():

    def __init__(self, frame):
        self.minimum_distance = 5

        self.lk_params = dict(
            winSize=(15,15),
            maxLevel=2,
            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03)
        )

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        h, w = gray.shape
        mask = np.zeros_like(gray)

        mask[:, :int(w*0.02)] = 1
        mask[:, int(w*0.9):] = 1

        self.features = dict(
            maxCorners=100,
            qualityLevel=0.3,
            minDistance=3,
            blockSize=7,
            mask=mask
        )

    def add_adjust_positions_to_tracks(self, tracks, camera_movement):

        for obj, obj_tracks in tracks.items():
            for frame_num, frame_tracks in enumerate(obj_tracks):
                for track_id, track_info in frame_tracks.items():

                    if "position" not in track_info:
                        continue

                    pos = track_info["position"]
                    move = camera_movement[frame_num]

                    adjusted = (pos[0]-move[0], pos[1]-move[1])

                    tracks[obj][frame_num][track_id]["position_adjusted"] = adjusted

    def get_camera_movement(self, frames, read_from_stub=False, stub_path=None):

        if read_from_stub and stub_path and os.path.exists(stub_path):
            with open(stub_path, "rb") as f:
                return pickle.load(f)

        movement = [[0,0] for _ in range(len(frames))]

        old_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
        old_features = cv2.goodFeaturesToTrack(old_gray, **self.features)

        for i in range(1, len(frames)):

            frame_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)

            new_features, status, _ = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, old_features, None, **self.lk_params
            )

            if new_features is None:
                continue

            dx = []
            dy = []

            for new, old in zip(new_features, old_features):
                n = new.ravel()
                o = old.ravel()

                dx.append(n[0]-o[0])
                dy.append(n[1]-o[1])

            if dx:
                movement[i] = [np.median(dx), np.median(dy)]

            old_gray = frame_gray.copy()
            old_features = cv2.goodFeaturesToTrack(frame_gray, **self.features)

        if stub_path:
            with open(stub_path, "wb") as f:
                pickle.dump(movement, f)

        return movement

    def draw_camera_movement(self, frames, camera_movement):

        output = []

        for i, frame in enumerate(frames):

            frame = frame.copy()

            x, y = camera_movement[i]

            cv2.putText(frame, f"Cam X: {x:.2f}", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 3)

            cv2.putText(frame, f"Cam Y: {y:.2f}", (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 3)

            output.append(frame)

        return output


In [51]:
# bounding box utils
def get_center_of_bbox(bbox):
    if bbox is None:
        return None

    x1, y1, x2, y2 = bbox

    if any(np.isnan([x1, y1, x2, y2])):
        return None

    return int((x1 + x2) / 2), int((y1 + y2) / 2)


def get_bbox_width(bbox):
    return bbox[2]-bbox[0]

def measure_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

def measure_xy_distance(p1,p2):
    return p1[0]-p2[0],p1[1]-p2[1]

def get_foot_position(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int(y2)



In [52]:
# reading and saving video
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise RuntimeError("Cannot open video")

    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    print(f"Loaded {len(frames)} frames")

    return frames


def save_video(output_video_frames, output_video_path, fps=25):
    if len(output_video_frames) == 0:
        raise ValueError("No frames to save")

    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    h, w, _ = output_video_frames[0].shape

    out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

    if not out.isOpened():
        raise RuntimeError("VideoWriter failed to open")

    for frame in output_video_frames:
        out.write(frame)

    out.release()
    print("Video saved at:", output_video_path)


In [53]:
def process_video_stream(video_path, output_path):

    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # ---- YOUR PROCESSING HERE ----
        processed = frame  # replace with pipeline step

        out.write(processed)

        frame_idx += 1

    cap.release()
    out.release()

    print("Done streaming processing")


In [54]:
class SpeedAndDistance_Estimator():
    def __init__(self):
        self.frame_window = 5
        self.frame_rate = 24

    def add_speed_and_distance_to_tracks(self, tracks):

        total_distance = {}

        for object_name, object_tracks in tracks.items():

            if object_name in ["ball", "referees"]:
                continue

            num_frames = len(object_tracks)

            for frame_num in range(0, num_frames, self.frame_window):

                last_frame = min(frame_num + self.frame_window, num_frames - 1)

                for track_id in object_tracks[frame_num].keys():

                    if track_id not in object_tracks[last_frame]:
                        continue

                    start_pos = object_tracks[frame_num][track_id].get("position_transformed")
                    end_pos = object_tracks[last_frame][track_id].get("position_transformed")

                    if start_pos is None or end_pos is None:
                        continue

                    distance = measure_distance(start_pos, end_pos)

                    frame_diff = last_frame - frame_num
                    if frame_diff <= 0:
                        continue

                    time_elapsed = frame_diff / self.frame_rate

                    speed_mps = distance / time_elapsed
                    speed_kmh = speed_mps * 3.6

                    if object_name not in total_distance:
                        total_distance[object_name] = {}

                    if track_id not in total_distance[object_name]:
                        total_distance[object_name][track_id] = 0

                    total_distance[object_name][track_id] += distance

                    for f in range(frame_num, last_frame):
                        if track_id not in tracks[object_name][f]:
                            continue

                        tracks[object_name][f][track_id]["speed"] = speed_kmh
                        tracks[object_name][f][track_id]["distance"] = total_distance[object_name][track_id]

    def draw_speed_and_distance(self, frames, tracks):

      output_frames = []

      for frame_num, frame in enumerate(frames):

          frame = frame.copy()

          for object_name, object_tracks in tracks.items():

              if object_name in ["ball", "referees"]:
                  continue

              for track_info in object_tracks[frame_num].values():

                  speed = track_info.get("speed")
                  distance = track_info.get("distance")

                  if speed is None or distance is None:
                      continue

                  bbox = track_info.get("bbox")
                  if bbox is None:
                      continue

                  pos = get_foot_position(bbox)
                  pos = (int(pos[0]), int(pos[1] + 40))

                  cv2.putText(frame, f"{speed:.1f} km/h", pos,
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

                  cv2.putText(frame, f"{distance:.1f} m", (pos[0], pos[1] + 20),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

          output_frames.append(frame)

      return output_frames


In [55]:
# assigning team
class TeamAssigner:
    def __init__(self):
        self.team_colors = {}
        self.player_team_dict = {}
        self.kmeans = None

    def get_clustering_model(self, image):
        image_2d = image.reshape(-1, 3)

        if len(image_2d) < 10:
            return None

        kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
        kmeans.fit(image_2d)

        return kmeans

    def get_player_color(self, frame, bbox):

        x1, y1, x2, y2 = map(int, bbox)

        h, w = frame.shape[:2]
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)

        image = frame[y1:y2, x1:x2]

        if image.size == 0:
            return np.array([0, 0, 0])

        top_half = image[: image.shape[0] // 2, :]

        model = self.get_clustering_model(top_half)

        if model is None:
            return np.array([0, 0, 0])

        labels = model.labels_
        clustered = labels.reshape(top_half.shape[0], top_half.shape[1])

        corners = [
            clustered[0, 0],
            clustered[0, -1],
            clustered[-1, 0],
            clustered[-1, -1],
        ]

        background_cluster = max(set(corners), key=corners.count)
        player_cluster = 1 - background_cluster

        return model.cluster_centers_[player_cluster]

    def assign_team_color(self, frame, player_detections):

        colors = []

        for det in player_detections.values():
            color = self.get_player_color(frame, det["bbox"])
            colors.append(color)

        if len(colors) < 2:
            return

        self.kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
        self.kmeans.fit(colors)

        self.team_colors[1] = tuple(map(int, self.kmeans.cluster_centers_[0]))
        self.team_colors[2] = tuple(map(int, self.kmeans.cluster_centers_[1]))

    def get_player_team(self, frame, player_bbox, player_id):

        if player_id in self.player_team_dict:
            return self.player_team_dict[player_id]

        if self.kmeans is None:
            return 1

        color = self.get_player_color(frame, player_bbox)

        team_id = self.kmeans.predict(color.reshape(1, -1))[0] + 1

        self.player_team_dict[player_id] = team_id

        return team_id


In [56]:
class PlayerBallAssigner():
    def __init__(self):
        self.max_player_ball_distance = 80  # slightly safer margin

    def assign_ball_to_player(self, players, ball_bbox):

        if ball_bbox is None or any(np.isnan(ball_bbox)):
            return -1

        ball_position = get_center_of_bbox(ball_bbox)

        if ball_position is None:
            return -1

        minimum_distance = float("inf")
        assigned_player = -1

        for player_id, player in players.items():

            player_bbox = player.get("bbox", None)
            if player_bbox is None or any(np.isnan(player_bbox)):
                continue

            player_position = get_foot_position(player_bbox)

            distance = measure_distance(player_position, ball_position)

            if distance < self.max_player_ball_distance and distance < minimum_distance:
                minimum_distance = distance
                assigned_player = player_id

        return assigned_player


In [57]:
class ViewTransformer():
    def __init__(self):

        # Real football pitch dimensions (meters)
        pitch_length = 105
        pitch_width = 68

        # Define manually selected corner points in image
        self.pixel_vertices = np.array([
            [110, 1035],
            [265, 275],
            [910, 260],
            [1640, 915]
        ], dtype=np.float32)

        # Target top-down plane
        self.target_vertices = np.array([
            [0, pitch_width],
            [0, 0],
            [pitch_length, 0],
            [pitch_length, pitch_width]
        ], dtype=np.float32)

        self.perspective_transformer = cv2.getPerspectiveTransform(
            self.pixel_vertices,
            self.target_vertices
        )

    def transform_point(self, point):

        if point is None:
            return None

        px = float(point[0])
        py = float(point[1])

        # Check if inside polygon
        inside = cv2.pointPolygonTest(self.pixel_vertices, (px, py), False)

        if inside < 0:
            return None

        pt = np.array([[[px, py]]], dtype=np.float32)
        transformed = cv2.perspectiveTransform(pt, self.perspective_transformer)

        return transformed.reshape(2).tolist()

    def add_transformed_position_to_tracks(self, tracks):

        for object_name, object_tracks in tracks.items():
            for frame_num, frame_tracks in enumerate(object_tracks):
                for track_id, track_info in frame_tracks.items():

                    if "position_adjusted" not in track_info:
                        continue

                    position = track_info["position_adjusted"]

                    transformed = self.transform_point(position)

                    tracks[object_name][frame_num][track_id]["position_transformed"] = transformed


# **TRACKING**

In [58]:
class Tracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()

    # Add position info to tracks
    def add_position_to_tracks(self, tracks):

        for object_name, object_tracks in tracks.items():
            for frame_num, frame_tracks in enumerate(object_tracks):
                for track_id, track_info in frame_tracks.items():
                    bbox = track_info["bbox"]
                    if bbox is None or any(np.isnan(bbox)):
                        continue
                    if object_name == "ball":
                        position = get_center_of_bbox(bbox)
                    else:
                        position = get_foot_position(bbox)
                    tracks[object_name][frame_num][track_id]["position"] = position

    # Interpolate missing ball positions
    def interpolate_ball_positions(self, ball_tracks):
        ball_positions = []

        for frame_ball in ball_tracks:
            if 1 in frame_ball and "bbox" in frame_ball[1]:
                ball_positions.append(frame_ball[1]["bbox"])
            else:
                ball_positions.append([np.nan, np.nan, np.nan, np.nan])

        df_ball = pd.DataFrame(
            ball_positions,
            columns=["x1", "y1", "x2", "y2"]
        )

        df_ball = df_ball.interpolate()
        df_ball = df_ball.bfill()

        new_ball_tracks = []
        for bbox in df_ball.to_numpy().tolist():
            new_ball_tracks.append({1: {"bbox": bbox}})

        return new_ball_tracks


    # Detect objects in frames
    def detect_frame(self, frames):
        batch_size = 5
        detection = []
        for i in range(0, len(frames), batch_size):
            batch = self.model.predict(frames[i:i + batch_size], conf=0.1)
            detection += batch
        return detection

    # Track players, referees and ball
    def track(self, frames, read_from_stub=False, stub_path=None):

          # Try loading stub safely
          if read_from_stub and stub_path and os.path.exists(stub_path):
              try:
                  with open(stub_path, "rb") as f:
                      track = pickle.load(f)
                  print("Loaded track stub")
                  return track
              except Exception as e:
                  print("Stub corrupted — recomputing:", e)

          # Otherwise compute tracking
          detection = self.detect_frame(frames)

          track = {
              "players": [],
              "referees": [],
              "ball": []
          }

          for frame_num, detection_frame in enumerate(detection):
              cls_name = detection_frame.names
              cls_name_inv = {v: k for k, v in cls_name.items()}

              detections_sv = sv.Detections.from_ultralytics(detection_frame)

              # Convert goalkeeper → player
              for idx, class_id in enumerate(detections_sv.class_id):
                  if cls_name[class_id] == "goalkeeper":
                      detections_sv.class_id[idx] = cls_name_inv["player"]

              detection_tracks = self.tracker.update_with_detections(detections_sv)

              track["players"].append({})
              track["referees"].append({})
              track["ball"].append({})

              # Players + referees
              for det in detection_tracks:
                  bbox = det[0].tolist()
                  cls_id = det[3]
                  track_id = det[4]

                  if cls_id == cls_name_inv["player"]:
                      track["players"][frame_num][track_id] = {"bbox": bbox}

                  if cls_id == cls_name_inv["referee"]:
                      track["referees"][frame_num][track_id] = {"bbox": bbox}

              # Ball selection — largest area
              best_ball = None
              best_area = 0

              for det in detections_sv:
                  bbox = det[0].tolist()
                  cls_id = det[3]

                  if cls_id == cls_name_inv["ball"]:
                      area = (bbox[2]-bbox[0]) * (bbox[3]-bbox[1])
                      if area > best_area:
                          best_area = area
                          best_ball = bbox

              if best_ball is not None:
                  track["ball"][frame_num][1] = {"bbox": best_ball}

          # Save stub
          if stub_path is not None:
              os.makedirs(os.path.dirname(stub_path), exist_ok=True)
              with open(stub_path, "wb") as f:
                  pickle.dump(track, f)

          return track


   # Draw ellipse for tracked objects
    def draw_ellipse(self, frame, bbox, color, track_id=None):
        y2 = int(bbox[3])
        x_center, _ = get_center_of_bbox(bbox)
        width = get_bbox_width(bbox)

        cv2.ellipse(
            frame,
            center=(x_center, y2),
            axes=(int(width), int(0.35 * width)),
            angle=0,
            startAngle=-45,
            endAngle=235,
            color=color,
            thickness=2,
            # lineType=cv2.LINE_4
        )

        if track_id is not None:
            cv2.putText(
                frame,
                str(track_id),
                (x_center - 10, y2 + 15),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 0, 0),
                2
            )

        return frame

    # Draw triangle for ball
    def draw_traingle(self, frame, bbox, color):
        y = int(bbox[1])
        x, _ = get_center_of_bbox(bbox)

        triangle = np.array([[x, y], [x - 10, y - 20], [x + 10, y - 20]])
        cv2.drawContours(frame, [triangle], 0, color, cv2.FILLED)
        cv2.drawContours(frame, [triangle], 0, (0, 0, 0), 2)

        return frame

    # Draw team ball control overlay
    def draw_team_ball_control(self, frame, frame_num, team_ball_control):
        overlay = frame.copy()
        cv2.rectangle(overlay, (1350, 850), (1900, 970), (255, 255, 255), -1)
        cv2.addWeighted(overlay, 0.4, frame, 0.6, 0, frame)

        tbc = team_ball_control[:frame_num + 1]

        t1 = sum(1 for x in tbc if x == 1)
        t2 = sum(1 for x in tbc if x == 2)


        total = t1 + t2 if (t1 + t2) > 0 else 1

        cv2.putText(frame, f"Team 1: {(t1 / total) * 100:.2f}%", (1400, 900),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
        cv2.putText(frame, f"Team 2: {(t2 / total) * 100:.2f}%", (1400, 950),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)

        return frame

    # Draw all annotations [for every individual we have different annotation]
    def draw_annotations(self, video_frames, tracks, team_ball_control):

        output_video_frames = []

        for frame_num, frame in enumerate(video_frames):

            frame = frame.copy()

            players = tracks["players"][frame_num]
            referees = tracks["referees"][frame_num]
            ball = tracks["ball"][frame_num]

            for track_id, player in players.items():
                bbox = player.get("bbox")
                if bbox is None:
                    continue

                color = player.get("team_color", (0,0,255))
                frame = self.draw_ellipse(frame, bbox, color, track_id)

            for _, referee in referees.items():
                bbox = referee.get("bbox")
                if bbox is None:
                    continue

                frame = self.draw_ellipse(frame, bbox, (0,255,255))

            for _, ball_info in ball.items():
                bbox = ball_info.get("bbox")
                if bbox is None:
                    continue

                frame = self.draw_traingle(frame, bbox, (0,255,0))

            frame = self.draw_team_ball_control(frame, frame_num, team_ball_control)

            output_video_frames.append(frame)

        return output_video_frames



In [59]:
cap = cv2.VideoCapture(video_path)
print(cap.isOpened())
cap = cv2.VideoCapture(video_path)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

print("Frame count:", frame_count)
print("FPS:", fps)


def main():

    print("===== STREAM PIPELINE START =====")

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise RuntimeError("Cannot open video")

    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    os.makedirs("/content/output", exist_ok=True)

    out = cv2.VideoWriter(
        "/content/output/output_video.mp4",
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h)
    )

    tracker = Tracker(model_path)

    # Load stub tracks once
    frames = read_video(video_path)
    tracks = tracker.track(frames, read_from_stub=True, stub_path=stub_path)
    tracker.add_position_to_tracks(tracks)

    camera_estimator = CameraMovementEstimator(frames[0])
    camera_movement = camera_estimator.get_camera_movement(
        frames,
        read_from_stub=True,
        stub_path="camera_movement_stub.pkl"
    )
    camera_estimator.add_adjust_positions_to_tracks(tracks, camera_movement)

    view_transformer = ViewTransformer()
    view_transformer.add_transformed_position_to_tracks(tracks)

    tracks["ball"] = tracker.interpolate_ball_positions(tracks["ball"])

    speed_estimator = SpeedAndDistance_Estimator()
    speed_estimator.add_speed_and_distance_to_tracks(tracks)

    team_ball_control = [0] * len(tracks["players"])

    print("Processing frames one by one...")

    frame_idx = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    while True:

        ret, frame = cap.read()
        if not ret:
            break

        frame = frame.copy()

        # Draw annotations
        annotated = tracker.draw_annotations(
            [frame],
            {
                "players":[tracks["players"][frame_idx]],
                "referees":[tracks["referees"][frame_idx]],
                "ball":[tracks["ball"][frame_idx]]
            },
            [team_ball_control[frame_idx]]
        )[0]

        # Camera overlay
        annotated = camera_estimator.draw_camera_movement(
            [annotated],
            [camera_movement[frame_idx]]
        )[0]

        # Speed overlay
        annotated = speed_estimator.draw_speed_and_distance(
            [annotated],
            {
                "players":[tracks["players"][frame_idx]],
                "referees":[tracks["referees"][frame_idx]],
                "ball":[tracks["ball"][frame_idx]]
            }
        )[0]

        out.write(annotated)

        frame_idx += 1

    cap.release()
    out.release()

    print("===== DONE — VIDEO SAVED =====")


if __name__ == "__main__":
    main()



True
Frame count: 750
FPS: 25.0
===== STREAM PIPELINE START =====
Loaded 750 frames

0: 384x640 3 balls, 22 players, 3 referees, 51.8ms
1: 384x640 3 balls, 22 players, 3 referees, 51.8ms
2: 384x640 2 balls, 1 goalkeeper, 23 players, 2 referees, 51.8ms
3: 384x640 1 ball, 23 players, 3 referees, 51.8ms
4: 384x640 1 ball, 22 players, 4 referees, 51.8ms
Speed: 7.2ms preprocess, 51.8ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 21 players, 3 referees, 31.0ms
1: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 31.0ms
2: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 31.0ms
3: 384x640 1 ball, 1 goalkeeper, 21 players, 3 referees, 31.0ms
4: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 31.0ms
Speed: 2.8ms preprocess, 31.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 34.9ms
1: 384x640 1 ball, 1 goalkeeper, 22 players, 3 referees, 34.9ms
2: 3